In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# Subquery

The first time you write a subquery it might seem really complex. Let's try breaking it down into its different parts.

If you get stuck look again at the video above. We want to find the average number of events for each day for each channel. The first table will provide us the number of events for each day and channel, and then we will need to average these values together using a second query.

In [6]:
pd.read_sql_query(sql='''
SELECT channel, AVG(event_count)
FROM (SELECT strftime('%Y-%m-%d', occurred_at) AS day,
channel,
COUNT (*) AS event_count
FROM web_events
GROUP BY 1, 2)
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,channel,AVG(event_count)
0,direct,4.896488
1,organic,1.667250
2,facebook,1.598347
3,adwords,1.570191
4,twitter,1.316667
5,banner,1.289973


SELECT channel, AVG(events) AS average_events
FROM (SELECT DATE_TRUNC('day',occurred_at) AS day,
             channel, COUNT(*) as events
      FROM web_events 
      GROUP BY 1,2) sub
GROUP BY channel
ORDER BY 2 DESC;

In [7]:
# subquery

pd.read_sql_query(sql='''
SELECT strftime('%Y-%m-%d', occurred_at) AS day,
channel,
COUNT (*) AS event_count
FROM web_events
GROUP BY 1, 2
ORDER BY 3 DESC;
''', con=conn)

,day,channel,event_count
0,2016-12-21,direct,21
1,2017-01-01,direct,21
2,2016-12-31,direct,19
3,2016-11-03,direct,18
4,2016-12-28,direct,17
...,...,...,...
3559,2016-12-30,twitter,1
3560,2016-12-31,adwords,1
3561,2016-12-31,banner,1
3562,2016-12-31,facebook,1


SELECT DATE_TRUNC('day', occurred_at) AS day,
channel,
COUNT (*) AS event_count
FROM web_events
GROUP BY 1, 2
ORDER BY 3 DESC